Assignment Date     : 23/10/2022 <br>
Student Name        : Arjun V <br>
Student Roll Number : 718020I461

In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn.metrics as m
import pickle

In [ ]:
df = pd.read_csv("/content/spam.csv",encoding = "ISO-8859-1")


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
labels=df.iloc[:,[0]]['v1']
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
labels=le.fit_transform(labels)
labels

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
from keras.utils.np_utils import to_categorical
labels=to_categorical(labels)
labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
sent=df.iloc[:,[1]]['v2']
sent

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [ ]:
import re
lemma=WordNetLemmatizer()
sentences=[]
for j in range(0,len(sent)):
  s=re.sub('[^A-Za-z]',' ',sent[j])
  s=s.lower()
  words=word_tokenize(s)
  words=[lemma.lemmatize(i) for i in words if i not in stopwords.words('english')]
  s=' '.join(words)
  sentences.append(s)

In [ ]:
sentences[:5]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though']

In [ ]:
from keras.preprocessing.text import one_hot
one_hot_sent=[one_hot(i,10000) for i in sentences]
maxi=[len(i) for i in one_hot_sent]
max(maxi)

77

In [ ]:
from keras_preprocessing.sequence import pad_sequences
padsequences=pad_sequences(one_hot_sent,maxlen=80)
padsequences

array([[   0,    0,    0, ..., 9338, 8026, 7320],
       [   0,    0,    0, ..., 2433, 6432,  369],
       [   0,    0,    0, ..., 7693, 5968, 9889],
       ...,
       [   0,    0,    0, ..., 7928, 1873, 6448],
       [   0,    0,    0, ..., 9275, 6432, 2416],
       [   0,    0,    0, ..., 9261, 2684, 7430]], dtype=int32)

In [ ]:
feature_train,feature_test,label_train,label_test=train_test_split(padsequences,labels,test_size=0.2,random_state=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout,Flatten
model=Sequential()
model.add(Embedding(10000,64,input_length=80))
model.add(LSTM(100))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 706,202
Trainable params: 706,202
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(feature_train,label_train,epochs=20,validation_data=(feature_test,label_test))

Epoch 1/20
140/140 [==============================] - 14s 83ms/step - loss: 0.2238 - accuracy: 0.9291 - val_loss: 0.0540 - val_accuracy: 0.9839
Epoch 2/20
140/140 [==============================] - 10s 72ms/step - loss: 0.0404 - accuracy: 0.9890 - val_loss: 0.0477 - val_accuracy: 0.9821
Epoch 3/20
140/140 [==============================] - 10s 72ms/step - loss: 0.0190 - accuracy: 0.9951 - val_loss: 0.0412 - val_accuracy: 0.9865
Epoch 4/20
140/140 [==============================] - 10s 73ms/step - loss: 0.0102 - accuracy: 0.9978 - val_loss: 0.0553 - val_accuracy: 0.9857
Epoch 5/20
140/140 [==============================] - 11s 76ms/step - loss: 0.0063 - accuracy: 0.9987 - val_loss: 0.0434 - val_accuracy: 0.9839
Epoch 6/20
140/140 [==============================] - 11s 81ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0543 - val_accuracy: 0.9848
Epoch 7/20
140/140 [==============================] - 10s 72ms/step - loss: 0.0023 - accuracy: 0.9996 - val_loss: 0.0517 - val_accuracy:

In [ ]:
label_pred=model.predict(feature_test)
label_pred

35/35 [==============================] - 1s 18ms/step


array([[9.9999434e-01, 5.9298582e-06],
       [5.9033591e-02, 9.3361908e-01],
       [9.9999672e-01, 3.5057626e-06],
       ...,
       [9.9997944e-01, 2.1895197e-05],
       [9.9998975e-01, 1.1403711e-05],
       [9.9999654e-01, 3.7449838e-06]], dtype=float32)

In [ ]:
label_pred_=[np.argmax(i,axis=0) for i in label_pred]
label_pred_[0:5]

[0, 1, 0, 0, 0]

In [ ]:
label_test_=[np.argmax(i,axis=0) for i in label_test]
label_test_[0:5]

[0, 0, 0, 0, 0]

In [ ]:
m.accuracy_score(label_test_,label_pred_)

0.9838565022421525

In [ ]:
print(m.classification_report(label_test_,label_pred_))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       970
           1       0.94      0.93      0.94       145

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
print(m.confusion_matrix(label_test_,label_pred_))

[[962   8]
 [ 10 135]]


In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
model.predict()